In [1]:
import sqlite3
import pandas as pd

In [2]:
from math import sqrt, atan2

In [3]:
NULL_SUBCARRIERS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 64, 65, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]

In [13]:
data = pd.read_csv("../training-data/Voxel-1/SittingDown-Movement-2024-04-11-13-50-48.csv")
data

,timestamp,rssi,csi,voxel_no,inter_voxel_no,activity
0,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 14,...",1,5,SittingDown-Movement
1,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, 0...",1,5,SittingDown-Movement
2,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, -4...",1,5,SittingDown-Movement
3,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -13, 7...",1,5,SittingDown-Movement
4,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -12, 9...",1,5,SittingDown-Movement
...,...,...,...,...,...,...
330,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 13,...",1,5,SittingDown-Movement
331,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -5, -1...",1,5,SittingDown-Movement
332,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 14...",1,5,SittingDown-Movement
333,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 13,...",1,5,SittingDown-Movement


In [14]:
data = data.iloc[:-5]

In [15]:
data

,timestamp,rssi,csi,voxel_no,inter_voxel_no,activity
0,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 14,...",1,5,SittingDown-Movement
1,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, 0...",1,5,SittingDown-Movement
2,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, -4...",1,5,SittingDown-Movement
3,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -13, 7...",1,5,SittingDown-Movement
4,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -12, 9...",1,5,SittingDown-Movement
...,...,...,...,...,...,...
325,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 5,...",1,5,SittingDown-Movement
326,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -15, -...",1,5,SittingDown-Movement
327,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -7, -1...",1,5,SittingDown-Movement
328,0,11,"[84, -64, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1...",1,5,SittingDown-Movement


In [16]:
# Remove the values in the array in the first column in CSI_DF whose indexes are given in the NULL_SUBCARRIERS array
# Convert the 'csi' column to a list of lists, assuming each entry is a string representation of a list
csi_df = data['csi'].apply(lambda x: eval(x))

# Define a function to remove null subcarriers
def remove_null_subcarriers(csi_list):
    return [value for index, value in enumerate(csi_list) if index not in NULL_SUBCARRIERS]

# Apply the function to each row in the 'csi' column
data['csi'] = csi_df.apply(remove_null_subcarriers)

/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_11366/1825363612.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['csi'] = csi_df.apply(remove_null_subcarriers)


In [17]:
for index, row in data.iterrows():
    csi = row['csi']
    subcarrier_index = 0
    for i in range(len(csi)):
        if i % 2 == 0:
            subcarrier_index += 1
            imaginary = csi[i]
            real = csi[i+1]
            ampltiude = sqrt(imaginary ** 2 + real ** 2)
            phase = atan2(imaginary, real)
            data.loc[index, f'subcarrier_{subcarrier_index}_amplitude'] = ampltiude
            data.loc[index, f'subcarrier_{subcarrier_index}_phase'] = phase
data

/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_11366/2749417376.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[index, f'subcarrier_{subcarrier_index}_amplitude'] = ampltiude
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_11366/2749417376.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[index, f'subcarrier_{subcarrier_index}_phase'] = phase
/var/folders/r8/m3g8wfxx51q2vc_tfm526t500000gn/T/ipykernel_11366/2749417376.py:11: SettingWithCopyWarning: 
A value is

,timestamp,rssi,csi,voxel_no,inter_voxel_no,activity,subcarrier_1_amplitude,subcarrier_1_phase,subcarrier_2_amplitude,subcarrier_2_phase,...,subcarrier_48_amplitude,subcarrier_48_phase,subcarrier_49_amplitude,subcarrier_49_phase,subcarrier_50_amplitude,subcarrier_50_phase,subcarrier_51_amplitude,subcarrier_51_phase,subcarrier_52_amplitude,subcarrier_52_phase
0,0,11,"[5, 14, 4, 16, 4, 16, 3, 17, 3, 16, 2, 16, 2, ...",1,5,SittingDown-Movement,14.866069,0.343024,16.492423,0.244979,...,21.213203,0.141897,20.396078,0.197396,20.880613,0.291457,20.396078,0.197396,19.416488,0.207496
1,0,11,"[-15, 0, -16, -1, -16, -1, -17, -2, -17, -2, -...",1,5,SittingDown-Movement,15.000000,-1.570796,16.031220,-1.633215,...,20.099751,-1.670465,21.095023,-1.665748,20.099751,-1.670465,20.099751,-1.670465,19.104973,-1.675673
2,0,11,"[14, -4, 14, -4, 14, -4, 16, -3, 16, -3, 15, -...",1,5,SittingDown-Movement,14.560220,1.849096,14.560220,1.849096,...,19.104973,1.675673,19.235384,1.727398,19.235384,1.727398,19.104973,1.675673,18.248288,1.735945
3,0,11,"[-13, 7, -14, 6, -14, 6, -15, 6, -16, 5, -16, ...",1,5,SittingDown-Movement,14.764823,-1.076855,15.231546,-1.165905,...,20.248457,-1.217806,19.313208,-1.199905,19.697716,-1.152572,20.248457,-1.217806,19.697716,-1.152572
4,0,11,"[-12, 9, -12, 9, -13, 8, -13, 8, -14, 8, -14, ...",1,5,SittingDown-Movement,15.000000,-0.927295,15.000000,-0.927295,...,19.723083,-1.039072,19.723083,-1.039072,19.416488,-0.968509,19.723083,-1.039072,19.416488,-0.968509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,0,11,"[14, 5, 15, 5, 15, 5, 16, 7, 15, 7, 15, 7, 14,...",1,5,SittingDown-Movement,14.866069,1.227772,15.811388,1.249046,...,20.591260,1.063698,19.723083,1.039072,19.697716,1.152572,20.591260,1.063698,20.124612,1.107149
326,0,11,"[-15, -1, -16, -1, -17, -1, -17, -2, -18, -3, ...",1,5,SittingDown-Movement,15.033296,-1.637364,16.031220,-1.633215,...,22.203603,-1.706324,22.360680,-1.750650,22.203603,-1.706324,21.377558,-1.759018,21.213203,-1.712693
327,0,11,"[-7, -15, -6, -15, -7, -16, -6, -17, -5, -18, ...",1,5,SittingDown-Movement,16.552945,-2.704965,16.155494,-2.761086,...,21.587033,-2.907849,22.803509,-2.875341,21.840330,-2.863293,21.587033,-2.907849,20.880613,-2.850136
328,0,11,"[-1, -17, -1, -16, 0, -19, 1, -18, 2, -19, 2, ...",1,5,SittingDown-Movement,17.029386,-3.082837,16.031220,-3.079174,...,23.345235,2.969402,25.317978,2.982937,24.186773,3.017238,24.515301,2.936197,23.345235,2.969402


In [18]:
# Calculate the number of subcarriers
num_subcarriers = (len(data.columns) - 6) // 2  # Subtracting the original columns and dividing by 2 for amplitude and phase

# Create a list to store the vectors
vectors_list = []

# Calculate the vectors for each subcarrier and add them to the list
for subcarrier in range(1, num_subcarriers + 1):
    # Create column names for amplitude and phase
    amplitude_col_name = f'subcarrier_{subcarrier}_amplitude'
    phase_col_name = f'subcarrier_{subcarrier}_phase'
    
    # Calculate the average for each block of 67 rows for amplitude and phase
    amplitude_vector = data[amplitude_col_name].groupby(data.index // 33).mean().reset_index(drop=True)
    phase_vector = data[phase_col_name].groupby(data.index // 33).mean().reset_index(drop=True)
    
    # Add the vectors as a new row in the list
    vectors_list.append({
        'subcarrier': subcarrier,
        'amplitude_vector': amplitude_vector.tolist(),
        'phase_vector': phase_vector.tolist()
    })

# Convert the list of vectors into a DataFrame
vectors_df = pd.DataFrame(vectors_list)

# vectors_df now contains rows for each subcarrier with amplitude and phase vectors

In [19]:
vectors_df.loc[0]["amplitude_vector"]

[15.035540307400504,
 14.848995793733947,
 14.650121146859087,
 15.154547004004712,
 14.569845023310966,
 14.487653477145663,
 14.940385561254091,
 14.720184989362624,
 14.52434013993702,
 14.528109110664094]

In [2]:
connection = sqlite3.connect("../database/rti.db")

In [3]:
cursor = connection.cursor()

In [8]:
#Create table to save threshold of occurence for each activity for each voxel for each subcarrier
cursor.execute("""
    CREATE TABLE IF NOT EXISTS distance_vector (
        voxel TEXT,
        subcarrier INTEGER,
        activity TEXT,
        amplitude_vector,
        phase_vector
    )
""")

In [ ]:
voxel_identifier = 1
activity_name = 'SittingDown-Movement'

# Iterate over the DataFrame and insert each vector into the SQL table
for index, row in vectors_df.iterrows():
    subcarrier = row['subcarrier']
    amplitude_vector = str(row['amplitude_vector'])  # Convert list to string to store in SQL
    phase_vector = str(row['phase_vector'])  # Convert list to string to store in SQL
    
    # Insert the data into the distance_vector table
    cursor.execute("""
        INSERT INTO distance_vector (voxel, subcarrier, activity, amplitude_vector, phase_vector)
        VALUES (?, ?, ?, ?, ?)
    """, (voxel_identifier, subcarrier, activity_name, amplitude_vector, phase_vector))

# Commit the transaction
connection.commit()